In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
from CloudDetect.util import read_off, visualise
from CloudDetect.transform import PointCloudSample, Normalise, Tensor
from CloudDetect.models import PointNet

In [153]:
import glob
metadata = pd.read_csv('data/metadata_modelnet40.csv')
all_data_files = glob.glob('**/*.off',recursive=True)
all_data_files = set(['/'.join(x.split('/')[2:]) for x in all_data_files])
metadata = metadata[metadata['object_path'].map(lambda x: x in all_data_files)]

In [154]:
file = ROOT + metadata['object_path'].iloc[-4465]
verts, faces = read_off(open(file))
#fig = visualise(verts)

new_verts = Normalise('max')(PointCloudSample(2000)(verts))
#fig = visualise(new_verts)

In [135]:
batch_size = 32
n_points = 2000
input = torch.tensor(np.random.uniform(0,1,(batch_size, 3, n_points)))

In [136]:
pnet = PointNet(n_points, classes = 10, segment = False)
classification_output = pnet(input.float())

In [122]:
# data loader

In [123]:
preprocessing(1024, 'max')(verts).shape

torch.Size([1024, 3])

In [155]:
def preprocessing(n_sample, norm_how):
    composition = [
            PointCloudSample(n_sample),
            Normalise(norm_how),
            Tensor()
    ]
    return transforms.Compose(composition)

def generate_class_mapper(metadata):
    class_mapper = {x:i for i,x in enumerate(metadata['class'].unique())}
    return class_mapper

In [156]:
import os
import pandas as pd
from torchvision.io import read_image

class CloudDataset(Dataset):
    def __init__(self, metadata, preprocessor, root, class_mapper):
        self.metadata = metadata
        self.preprocessor = preprocessor
        self.root = root
        self.class_mapper = class_mapper
        
    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        data_instance = metadata.iloc[idx]
        class_item = self.class_mapper[data_instance['class']]
        class_item = torch.nn.functional.one_hot(torch.tensor(class_item), num_classes=len(self.class_mapper))
        file = self.root + data_instance['object_path']
        verts, faces = read_off(open(file))
        return {'data':self.preprocessor(verts),'category':class_item}

In [157]:
ROOT = 'data/ModelNet40/'
preprocessor = preprocessing(1024, 'max')
class_mapper = generate_class_mapper(metadata)
cloud_dataset = CloudDataset(metadata, preprocessor, ROOT, class_mapper)

In [158]:
CloudDataLoader = DataLoader(cloud_dataset, batch_size=64, shuffle=True)

In [159]:
for x in CloudDataLoader:
    break